In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import networkx as nx
import os
import folium
import branca.colormap as cm
import geopy.distance as gpd
from scipy.spatial.distance import pdist, squareform
import json

# Functions

In [2]:
def get_df_in_times(df, start_time, end_time):
    return df[(df.timestamp > start_time) & (df.timestamp < end_time)]

In [3]:
def get_timestamps(df):
    timestamps = df.timestamp.unique()
    hourly_timestamps = [timestamps[-i] for i in range(len(timestamps)) if i%6==1]
    # This gives ASCENDING order 4:55am - 5:55am - .... 2:55am - 3:55am
    # Do the same for ten_minute
    ten_minute_timestamps = timestamps[::-1]
    
    return hourly_timestamps, ten_minute_timestamps

In [4]:
def geodesic_distance(a, b):
    return gpd.distance(a, b).m

In [5]:
def build_maps(df):
    temp_df = df[df.timestamp == df.timestamp.unique()[0]]  # just need to get the location of each station
    temp_df = temp_df['id'].reset_index()

    # Used reset_index for the iterrows() function, which we want to start counting at 0
    node_to_station_map = {}  # {node_id : station_id}
    station_to_node_map = {}  # {station_id : node_id}
    for index, row in temp_df.iterrows():
        node_to_station_map[index] = int(row.id)
        station_to_node_map[int(row.id)] = index

    return node_to_station_map, station_to_node_map

In [6]:
def build_graph(df, station_to_node_map):
    # Warning: quite slow (pdist)
    temp_df = df[df.timestamp == df.timestamp.unique()[0]]  # just need to get the location of each station
    temp_df = temp_df[['id','latitude','longitude']]
    # Used reset_index for the iterrows() function, which we want to start counting at 0
    matrix = np.empty((temp_df.shape[0], 2))

    for index, row in temp_df.iterrows():
        station_id = row.id  # id of station, needs mapping to id of node
        node_id = station_to_node_map[station_id]
        matrix[node_id,:] = [row.latitude, row.longitude]
    
    distance_matrix = pdist(matrix, geodesic_distance)
    distance_matrix = squareform(distance_matrix)
    
    return distance_matrix

In [7]:
def get_station_states(dframe, unique_timestamps):
    # Make a dictionary of the 'state' of each station for each timestamp
    states = {}
#     timestamp_dict = {}
    for i, timestamp in enumerate(unique_timestamps):  # Should be in ASCENDING order
        temp_df = dframe[dframe.timestamp == timestamp]
        states[i] = {}
        percentages = {}
        for index, row in temp_df.iterrows():
            percentages[row.id] = row.percent_full  # {station_id : percentage}

        states[i]['timestamp'] = timestamp.astype(str)
        states[i]['percentages'] = percentages  
    
    return states

In [8]:
def save_station_states_to_json(states, resolution, mode):
    if mode == 'train':
        filename = 'train_station_{}_states.json'.format(resolution)
    elif mode == 'test':
        filename = 'test_station_{}_states.json'.format(resolution)
    elif mode == 'mini_train':
        filename = 'mini_train_station_{}_states.json'.format(resolution)
    elif mode == 'mini_test':
        filename = 'mini_test_station_{}_states.json'.format(resolution)
    elif mode == 'mini_total':
        filename = 'mini_total_station_{}_states.json'.format(resolution)
    
    with open(filename, 'w') as fp:
        json.dump(states, fp)

In [9]:
def generate_binary_labelings(station_states, station_to_node_map, percentage_threshold):
    labelings = {}

    for segment, data in station_states.items():
        labeling = {}
        for station, percentage in data['percentages'].items():
            node = station_to_node_map[station]
            labeling[node] = int(int(percentage >= percentage_threshold)*2 - 1)  #+1 or -1

        labelings[int(segment)] = labeling

    return labelings

In [10]:
def get_ids_of_switching_nodes(labelings):
    switching_nodes = set()

    previous_labeling = labelings[0]

    for segment, labeling in labelings.items():
        if segment == 0:  # skip first labeling
            continue
        for node, label in labeling.items():  # The labeling has the form {node_id: label}
            if label != previous_labeling[node]:
                switching_nodes.add(node)

        previous_labeling = labeling

    return list(switching_nodes)

In [11]:
def build_knn_matrix(distance_matrix, k):
    argsorted = np.argsort(distance_matrix.view(np.ndarray), axis=1)[:,::1]

    knn = np.zeros_like(distance_matrix, dtype=np.int8)

    for i in range(argsorted.shape[0]):
        for j in argsorted[i,:k]:
            knn[i,j] = 1
        knn[i,i] = 0
    
    return knn

In [12]:
def plot_labels_on_map(df, timestamps, threshold, location, color1, color2, radius):
    mean_lat = np.mean(df['latitude'])
    mean_long = np.mean(df['longitude'])
    for timestamp in timestamps:
        temp_df = df[df.timestamp == timestamp]
        mapit = folium.Map( location=[ mean_lat, mean_long ], zoom_start=11 )
        for index, row in temp_df.iterrows():
            lat = row['latitude']
            long = row['longitude']
            if row['percent_full'] < threshold:
                color=color1
            else:
                color=color2
            folium.CircleMarker(radius=radius, location=[ lat, long], color=color ).add_to(mapit)
        mapit.save( os.path.join(location,'{}.html'.format(str(timestamp))))

# Load all data, get 3 days, save to file (ONLY DO ONCE)

In [13]:
data_dir = 'rawdata'

train_start_time = np.datetime64('2019-04-08T04:50:00')
train_end_time = np.datetime64('2019-04-09T04:50:00')
test_start_time = train_end_time
test_end_time = np.datetime64('2019-04-11T04:50:00')

In [14]:
# with open('rawdata/chicago_april.json', 'r') as f:
#     data = json.load(f)

# train_and_test = []
# train = []
# test = []
# for datum in data:
#     dt = np.datetime64(datum['timestamp'])
#     # add to train
#     if dt >= train_start_time and dt < train_end_time:
#         train.append(datum)
#     # add to test
#     if dt >= test_start_time and dt < test_end_time:
#         test.append(datum)
#     # add to train_and_test
#     if dt >= train_start_time and dt < test_end_time:
#         train_and_test.append(datum)

# # Save json files
# with open('rawdata/train.json', 'w') as f:
#     json.dump(train, f)
# with open('rawdata/test.json', 'w') as f:
#     json.dump(test, f)
# with open('rawdata/train_and_test.json', 'w') as f:
#     json.dump(train_and_test, f)
    

There are 608 bike stations. 

The timestamps are every 10 minutes (roughly). We will use both ten-minute and hourly data, so for the latter we want to take every seventh timestamp.

In [15]:
# sns.set(color_codes=True)
# plt.title('Distribution of percentage states of stations')
# sns.distplot(df['percent_full'],bins=20)
# plt.show()

# Load 3 days of data, get timestamps and generate node ids

3 consecutive days of training data. 
This has already been processed from the big json file, and has been saved into 3 json files:

train.json - just the 24 hour train data

test.json - just the 48 hour test data

train_and_test.json - all 72 hours of data



In [16]:
# Load the dataframes from the csv files 
df_total = pd.read_json(os.path.join(data_dir, 'train_and_test.json'))
df_train = pd.read_json(os.path.join(data_dir, 'train.json'))
df_test = pd.read_json(os.path.join(data_dir, 'test.json'))

# Get the 3 day timestamps
total_hourly_timestamps, total_ten_minute_timestamps = get_timestamps(df_total)
# Get the train timestamps
train_hourly_timestamps, train_ten_minute_timestamps = get_timestamps(df_train)
# Get the test timestamps
test_hourly_timestamps, test_ten_minute_timestamps = get_timestamps(df_test)

In [17]:
df_total.timestamp.nunique()

432

In [18]:
df_total.status.unique()

array(['In Service', 'Not In Service'], dtype=object)

In [19]:
# Remove the station which has some 'Not in service issues'
bad_id = df_total[df_total.status == 'Not In Service'].id.unique()[0]

df_total = df_total[df_total.id != bad_id]
df_train = df_train[df_train.id != bad_id]
df_test = df_test[df_test.id != bad_id]

Now we need to convert this data into binary labelings, so that we can see which nodes switch and which don't.
First we build node-station and station-node mappings, then get a dictionary of the state (percentage full) of each station on each timestamp, then turn those into binary labelings

In [20]:
# Build the node-station and station-node dictionaries
node_to_station_map, station_to_node_map = build_maps(df_total)

# Get the station_states dictionary for 3 days data
total_station_states = get_station_states(df_total, total_ten_minute_timestamps)

# Turn this into a binarly labeling
percentage_threshold = 50
total_binary_labelings = generate_binary_labelings(station_states=total_station_states,
                                                 station_to_node_map=station_to_node_map,
                                                 percentage_threshold=percentage_threshold)

In [21]:
# From the binary labelings, get the ids of the ndoes that switch at some point
total_switching_node_ids = get_ids_of_switching_nodes(total_binary_labelings)
total_switching_station_ids = [node_to_station_map[node_id] for node_id in total_switching_node_ids]

Now we have the ids of the nodes which switch during the 3 days, we need to create:

new dataframes (train, test)

new station_to_node_map and node_to_station_map

states 

distance matrix

knn graph

In [22]:
# New dataframes
mini_df_train = df_train[df_train.id.isin(total_switching_station_ids)]
mini_df_test = df_test[df_test.id.isin(total_switching_station_ids)]

# New maps
mini_node_to_station_map, mini_station_to_node_map = build_maps(mini_df_train)

# New states
mini_train_hourly_states = get_station_states(mini_df_train, train_hourly_timestamps)
mini_train_ten_minute_states = get_station_states(mini_df_train, train_ten_minute_timestamps)
mini_test_hourly_states = get_station_states(mini_df_test, test_hourly_timestamps)
mini_test_ten_minute_states = get_station_states(mini_df_test, test_ten_minute_timestamps)

# Distance matrix
mini_distance_matrix = build_graph(mini_df_train, mini_station_to_node_map)

In [23]:
# Save all the data in one for spine checks
mini_df_total = df_total[df_total.id.isin(total_switching_station_ids)]
mini_total_ten_minute_states = get_station_states(mini_df_total, total_ten_minute_timestamps)
# Save test data (ten_minute)
save_station_states_to_json(states=mini_total_ten_minute_states,
                            resolution='ten_minute',
                            mode='mini_total')

# Plot the train/test labelings on the map

In [23]:
plot_labels_on_map(df=mini_df_train,
                   timestamps=train_ten_minute_timestamps,
                   threshold=50,
                   location='plots/train',
                   color1='orange',
                   color2='black',
                   radius=2)

# Save the useful data to disk

In [25]:
# This method considers self-neighboring, which we will remove later so really k=3
k = 4
knn = build_knn_matrix(distance_matrix=mini_distance_matrix, k=k)
np.save('mini_{}_knn_matrix'.format(k-1), knn)

In [26]:
# Save the mini data
# Save the pair-wise discance matrix 
np.save('mini_station_distance_matrix', mini_distance_matrix)

# Save the station to node map { station_id : node_id }
with open('mini_station_to_node_map.json', 'w') as fp:
    json.dump(mini_station_to_node_map, fp)

# Save the node to station map { node_id : station_id }
with open('mini_node_to_station_map.json', 'w') as fp:
    json.dump(mini_node_to_station_map, fp)
    
# Save training data (hourly)
save_station_states_to_json(states=mini_train_hourly_states, 
                            resolution='hourly',
                            mode='mini_train')

# Save training data (ten_minute)
save_station_states_to_json(states=mini_train_ten_minute_states,
                            resolution='ten_minute',
                            mode='mini_train')

# Save test data (hourly)
save_station_states_to_json(states=mini_test_hourly_states,
                            resolution='hourly',
                            mode='mini_test')

# Save test data (ten_minute)
save_station_states_to_json(states=mini_test_ten_minute_states,
                            resolution='ten_minute',
                            mode='mini_test')
